In [40]:
import nltk
from nltk.corpus import stopwords

# Baixar recursos do NLTK
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Rafael\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rafael\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [42]:
import PyPDF2

# Função para extrair texto de PDFs
def pdf_para_txt(caminho_pdf):
    with open(caminho_pdf, "rb") as f:
        leitor = PyPDF2.PdfReader(f)
        texto = ""
        for pagina in range(len(leitor.pages)):
            texto += leitor.pages[pagina].extract_text()
    return texto

# Diretórios com os PDFs
diretorios = {
    'poesia': 'pdfs/poesia/',
    'prosa': 'pdfs/prosa/',
    'jornalismo': 'pdfs/jornalismo/'
}

# Função para limpar e remover stopwords
def limpar_texto(texto):
    texto = texto.replace("\n", " ")
    stop_words = set(stopwords.words("english"))
    palavras = nltk.word_tokenize(texto.lower())
    palavras_limpa = [palavra for palavra in palavras if palavra.isalnum() and palavra not in stop_words]
    return " ".join(palavras_limpa)

import os

# Extraindo textos e gerando classes
textos = []
classes = []

for classe, caminho in diretorios.items():
    for arquivo in os.listdir(caminho):
        if arquivo.endswith(".pdf"):
            texto = pdf_para_txt(os.path.join(caminho, arquivo))
            texto_limpo = limpar_texto(texto)
            textos.append(texto_limpo)
            classes.append(classe)


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Criando a matriz Bag of Words
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(textos)

print("Matriz BoW: ", X.toarray())
print("Vocabulário: ", vectorizer.get_feature_names_out())
print("Classes: ", classes)

Matriz BoW:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Vocabulário:  ['00' '01' '02' ... '⁹¹the' '⁹⁰but' '⁹⁰no']
Classes:  ['poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'poesia', 'p

In [44]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score

# Criando o modelo base
modelo = DecisionTreeClassifier()

# Configuração de hiperparâmetros para teste
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 1, 3]
}

# Métricas para otimização
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_weighted': make_scorer(f1_score, average='weighted'),
}

# Configurando o GridSearchCV com validação cruzada
grid_search = GridSearchCV(
    estimator=modelo,
    param_grid=param_grid,
    scoring=scoring,
    refit='accuracy',
    cv=10,  # Validação cruzada estratificada com 10 folds
    verbose=2,
    n_jobs=-1,
)

# Treinando o modelo com GridSearch
grid_search.fit(X, classes)

# Exibindo os melhores hiperparâmetros e resultados
print("Melhores hiperparâmetros para Árvore de decisão:", grid_search.best_params_)
print(f"Melhor acurácia: {grid_search.best_score_:.4f}")

# Obtendo os resultados detalhados
resultados = grid_search.cv_results_

# Iterar sobre cada combinação de hiperparâmetros
for i in range(len(resultados['params'])):
    print(f"Configuração de Hiperparâmetros {i+1}: {resultados['params'][i]}")

    # Métricas para cada fold
    for fold in range(10):  # 10 folds
        acuracia_fold = resultados[f'split{fold}_test_accuracy'][i]
        f1_fold = resultados[f'split{fold}_test_f1_weighted'][i]
        print(f"  Fold {fold+1}: Acurácia = {acuracia_fold:.4f}, F1-score = {f1_fold:.4f}")

    # Métricas médias
    acuracia_media = resultados['mean_test_accuracy'][i]
    f1_media = resultados['mean_test_f1_weighted'][i]
    acuracia_std = resultados['std_test_accuracy'][i]
    f1_std = resultados['std_test_f1_weighted'][i]

    print(f"  Acurácia média: {acuracia_media:.4f}, Desvio padrão: {acuracia_std:.4f}")
    print(f"  F1-score médio: {f1_media:.4f}, Desvio padrão: {f1_std:.4f}")
    print("-" * 50)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Melhores hiperparâmetros para Árvore de decisão: {'criterion': 'entropy', 'max_depth': 3, 'splitter': 'best'}
Melhor acurácia: 0.9667
Configuração de Hiperparâmetros 1: {'criterion': 'gini', 'max_depth': None, 'splitter': 'best'}
  Fold 1: Acurácia = 0.9333, F1-score = 0.9333
  Fold 2: Acurácia = 0.9333, F1-score = 0.9333
  Fold 3: Acurácia = 1.0000, F1-score = 1.0000
  Fold 4: Acurácia = 0.9667, F1-score = 0.9666
  Fold 5: Acurácia = 0.9333, F1-score = 0.9333
  Fold 6: Acurácia = 0.8333, F1-score = 0.8222
  Fold 7: Acurácia = 0.9667, F1-score = 0.9666
  Fold 8: Acurácia = 0.9667, F1-score = 0.9666
  Fold 9: Acurácia = 1.0000, F1-score = 1.0000
  Fold 10: Acurácia = 1.0000, F1-score = 1.0000
  Acurácia média: 0.9533, Desvio padrão: 0.0476
  F1-score médio: 0.9522, Desvio padrão: 0.0504
--------------------------------------------------
Configuração de Hiperparâmetros 2: {'criterion': 'gini', 'max_depth': None, 'splitter': '

In [45]:
from sklearn.neighbors import KNeighborsClassifier

# Criando o modelo base
modelo = KNeighborsClassifier()

# Configuração de hiperparâmetros para teste
param_grid = {
    'n_neighbors': [2, 5, 7],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree']
}

# Configurando o GridSearchCV com validação cruzada
grid_search = GridSearchCV(
    estimator=modelo,
    param_grid=param_grid,
    scoring=scoring,
    refit='accuracy',
    cv=10,  # Validação cruzada estratificada com 10 folds
    verbose=2,
    n_jobs=-1,
)

# Treinando o modelo com GridSearch
grid_search.fit(X, classes)

# Exibindo os melhores hiperparâmetros e resultados
print("Melhores hiperparâmetros para KNN:", grid_search.best_params_)
print(f"Melhor acurácia: {grid_search.best_score_:.4f}")

# Obtendo os resultados detalhados
resultados = grid_search.cv_results_

# Iterar sobre cada combinação de hiperparâmetros
for i in range(len(resultados['params'])):
    print(f"Configuração de Hiperparâmetros {i+1}: {resultados['params'][i]}")

    # Métricas para cada fold
    for fold in range(10):  # 10 folds
        acuracia_fold = resultados[f'split{fold}_test_accuracy'][i]
        f1_fold = resultados[f'split{fold}_test_f1_weighted'][i]
        print(f"  Fold {fold+1}: Acurácia = {acuracia_fold:.4f}, F1-score = {f1_fold:.4f}")

    # Métricas médias
    acuracia_media = resultados['mean_test_accuracy'][i]
    f1_media = resultados['mean_test_f1_weighted'][i]
    acuracia_std = resultados['std_test_accuracy'][i]
    f1_std = resultados['std_test_f1_weighted'][i]

    print(f"  Acurácia média: {acuracia_media:.4f}, Desvio padrão: {acuracia_std:.4f}")
    print(f"  F1-score médio: {f1_media:.4f}, Desvio padrão: {f1_std:.4f}")
    print("-" * 50)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Melhores hiperparâmetros para KNN: {'algorithm': 'auto', 'n_neighbors': 7, 'weights': 'uniform'}
Melhor acurácia: 0.9733
Configuração de Hiperparâmetros 1: {'algorithm': 'auto', 'n_neighbors': 2, 'weights': 'uniform'}
  Fold 1: Acurácia = 0.9667, F1-score = 0.9666
  Fold 2: Acurácia = 0.9000, F1-score = 0.8977
  Fold 3: Acurácia = 0.9333, F1-score = 0.9327
  Fold 4: Acurácia = 0.9333, F1-score = 0.9327
  Fold 5: Acurácia = 0.8667, F1-score = 0.8611
  Fold 6: Acurácia = 1.0000, F1-score = 1.0000
  Fold 7: Acurácia = 0.9667, F1-score = 0.9666
  Fold 8: Acurácia = 0.9333, F1-score = 0.9327
  Fold 9: Acurácia = 1.0000, F1-score = 1.0000
  Fold 10: Acurácia = 1.0000, F1-score = 1.0000
  Acurácia média: 0.9500, Desvio padrão: 0.0428
  F1-score médio: 0.9490, Desvio padrão: 0.0442
--------------------------------------------------
Configuração de Hiperparâmetros 2: {'algorithm': 'auto', 'n_neighbors': 2, 'weights': 'distance'}
  F

In [47]:
from sklearn.naive_bayes import MultinomialNB

# Criando o modelo base
modelo = MultinomialNB()

# Configuração de hiperparâmetros para teste
param_grid = {
    'alpha': [0.1, 0.5, 1.0],
    'fit_prior': [True, False]
}

# Configurando o GridSearchCV com validação cruzada
grid_search = GridSearchCV(
    estimator=modelo,
    param_grid=param_grid,
    scoring=scoring,
    refit='accuracy',
    cv=10,  # Validação cruzada estratificada com 10 folds
    verbose=2,
    n_jobs=-1,
)

# Treinando o modelo com GridSearch
grid_search.fit(X, classes)

# Exibindo os melhores hiperparâmetros e resultados
print("Melhores hiperparâmetros para Naive Bayes:", grid_search.best_params_)
print(f"Melhor acurácia: {grid_search.best_score_:.4f}")

# Obtendo os resultados detalhados
resultados = grid_search.cv_results_

# Iterar sobre cada combinação de hiperparâmetros
for i in range(len(resultados['params'])):
    print(f"Configuração de Hiperparâmetros {i+1}: {resultados['params'][i]}")

    # Métricas para cada fold
    for fold in range(10):  # 10 folds
        acuracia_fold = resultados[f'split{fold}_test_accuracy'][i]
        f1_fold = resultados[f'split{fold}_test_f1_weighted'][i]
        print(f"  Fold {fold+1}: Acurácia = {acuracia_fold:.4f}, F1-score = {f1_fold:.4f}")

    # Métricas médias
    acuracia_media = resultados['mean_test_accuracy'][i]
    f1_media = resultados['mean_test_f1_weighted'][i]
    acuracia_std = resultados['std_test_accuracy'][i]
    f1_std = resultados['std_test_f1_weighted'][i]

    print(f"  Acurácia média: {acuracia_media:.4f}, Desvio padrão: {acuracia_std:.4f}")
    print(f"  F1-score médio: {f1_media:.4f}, Desvio padrão: {f1_std:.4f}")
    print("-" * 50)

Fitting 10 folds for each of 6 candidates, totalling 60 fits
Melhores hiperparâmetros para Naive Bayes: {'alpha': 0.1, 'fit_prior': True}
Melhor acurácia: 0.9400
Configuração de Hiperparâmetros 1: {'alpha': 0.1, 'fit_prior': True}
  Fold 1: Acurácia = 0.9333, F1-score = 0.9327
  Fold 2: Acurácia = 0.9333, F1-score = 0.9327
  Fold 3: Acurácia = 0.9000, F1-score = 0.8977
  Fold 4: Acurácia = 0.9000, F1-score = 0.8977
  Fold 5: Acurácia = 0.9333, F1-score = 0.9327
  Fold 6: Acurácia = 0.9667, F1-score = 0.9666
  Fold 7: Acurácia = 0.9667, F1-score = 0.9666
  Fold 8: Acurácia = 0.9333, F1-score = 0.9327
  Fold 9: Acurácia = 0.9667, F1-score = 0.9666
  Fold 10: Acurácia = 0.9667, F1-score = 0.9666
  Acurácia média: 0.9400, Desvio padrão: 0.0249
  F1-score médio: 0.9392, Desvio padrão: 0.0257
--------------------------------------------------
Configuração de Hiperparâmetros 2: {'alpha': 0.1, 'fit_prior': False}
  Fold 1: Acurácia = 0.9333, F1-score = 0.9327
  Fold 2: Acurácia = 0.9333, F1-sc

In [48]:
from sklearn.linear_model import LogisticRegression

# Criando o modelo base
modelo = LogisticRegression()

# Configuração de hiperparâmetros para teste
param_grid = {
    'solver': ['lbfgs', 'saga', 'liblinear'],
    'class_weight': [None, 'balanced'],
}

# Configurando o GridSearchCV com validação cruzada
grid_search = GridSearchCV(
    estimator=modelo,
    param_grid=param_grid,
    scoring=scoring,
    refit='accuracy',
    cv=10,  # Validação cruzada estratificada com 10 folds
    verbose=2,
    n_jobs=-1,
)

# Treinando o modelo com GridSearch
grid_search.fit(X, classes)

# Exibindo os melhores hiperparâmetros e resultados
print("Melhores hiperparâmetros para Regressão logística:", grid_search.best_params_)
print(f"Melhor acurácia: {grid_search.best_score_:.4f}")

# Obtendo os resultados detalhados
resultados = grid_search.cv_results_

# Iterar sobre cada combinação de hiperparâmetros
for i in range(len(resultados['params'])):
    print(f"Configuração de Hiperparâmetros {i+1}: {resultados['params'][i]}")

    # Métricas para cada fold
    for fold in range(10):
        acuracia_fold = resultados[f'split{fold}_test_accuracy'][i]
        f1_fold = resultados[f'split{fold}_test_f1_weighted'][i]
        print(f"  Fold {fold+1}: Acurácia = {acuracia_fold:.4f}, F1-score = {f1_fold:.4f}")

    # Métricas médias
    acuracia_media = resultados['mean_test_accuracy'][i]
    f1_media = resultados['mean_test_f1_weighted'][i]
    acuracia_std = resultados['std_test_accuracy'][i]
    f1_std = resultados['std_test_f1_weighted'][i]

    print(f"  Acurácia média: {acuracia_media:.4f}, Desvio padrão: {acuracia_std:.4f}")
    print(f"  F1-score médio: {f1_media:.4f}, Desvio padrão: {f1_std:.4f}")
    print("-" * 50)

Fitting 10 folds for each of 6 candidates, totalling 60 fits
Melhores hiperparâmetros para Regressão logística: {'class_weight': None, 'solver': 'lbfgs'}
Melhor acurácia: 0.9733
Configuração de Hiperparâmetros 1: {'class_weight': None, 'solver': 'lbfgs'}
  Fold 1: Acurácia = 0.9667, F1-score = 0.9666
  Fold 2: Acurácia = 0.9667, F1-score = 0.9666
  Fold 3: Acurácia = 1.0000, F1-score = 1.0000
  Fold 4: Acurácia = 0.9667, F1-score = 0.9666
  Fold 5: Acurácia = 1.0000, F1-score = 1.0000
  Fold 6: Acurácia = 0.8667, F1-score = 0.8611
  Fold 7: Acurácia = 1.0000, F1-score = 1.0000
  Fold 8: Acurácia = 0.9667, F1-score = 0.9666
  Fold 9: Acurácia = 1.0000, F1-score = 1.0000
  Fold 10: Acurácia = 1.0000, F1-score = 1.0000
  Acurácia média: 0.9733, Desvio padrão: 0.0389
  F1-score médio: 0.9727, Desvio padrão: 0.0404
--------------------------------------------------
Configuração de Hiperparâmetros 2: {'class_weight': None, 'solver': 'saga'}
  Fold 1: Acurácia = 0.9667, F1-score = 0.9666
  Fo

In [ ]:
from sklearn.neural_network import MLPClassifier

# Criando o modelo base
modelo = MLPClassifier()

# Configuração de hiperparâmetros para teste
param_grid = {
    "hidden_layer_sizes": [(50,), (100,)],
    'activation': ['relu', 'tanh'],
    'early_stopping': [True],
}

# Configurando o GridSearchCV com validação cruzada
grid_search = GridSearchCV(
    estimator=modelo,
    param_grid=param_grid,
    scoring=scoring,
    refit='accuracy',
    cv=10,  # Validação cruzada estratificada com 10 folds
    verbose=2,
    n_jobs=-1,
)

# Treinando o modelo com GridSearch
grid_search.fit(X, classes)

# Exibindo os melhores hiperparâmetros e resultados
print("Melhores hiperparâmetros para Rede Neural MLP:", grid_search.best_params_)
print(f"Melhor acurácia: {grid_search.best_score_:.4f}")

# Obtendo os resultados detalhados
resultados = grid_search.cv_results_

# Iterar sobre cada combinação de hiperparâmetros
for i in range(len(resultados['params'])):
    print(f"Configuração de Hiperparâmetros {i+1}: {resultados['params'][i]}")

    # Métricas para cada fold
    for fold in range(10):  # 10 folds
        acuracia_fold = resultados[f'split{fold}_test_accuracy'][i]
        f1_fold = resultados[f'split{fold}_test_f1_weighted'][i]
        print(f"  Fold {fold+1}: Acurácia = {acuracia_fold:.4f}, F1-score = {f1_fold:.4f}")

    # Métricas médias
    acuracia_media = resultados['mean_test_accuracy'][i]
    f1_media = resultados['mean_test_f1_weighted'][i]
    acuracia_std = resultados['std_test_accuracy'][i]
    f1_std = resultados['std_test_f1_weighted'][i]

    print(f"  Acurácia média: {acuracia_media:.4f}, Desvio padrão: {acuracia_std:.4f}")
    print(f"  F1-score médio: {f1_media:.4f}, Desvio padrão: {f1_std:.4f}")
    print("-" * 50)


Fitting 10 folds for each of 8 candidates, totalling 80 fits


KeyboardInterrupt: 